# Comparar catálogo

## Parámetros

In [2]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Catalog/Decathlon'
FILE_MX = 'Decathlon_MX.csv'
FILE_CL = 'Decathlon_CL.csv'

## Clase

In [35]:
# Control de datos
from time import sleep
from pathlib import Path
from IPython.display import clear_output

# Ingeniería de variables
from numpy import nan
from re import sub, UNICODE
from unicodedata import normalize
from difflib import get_close_matches
from pandas import DataFrame, read_csv, options
options.mode.chained_assignment = None

class CompareCatalog:
    def __init__(self, base_dir:str, file_mx: str, file_cl: str) -> None:
        '''
        Obtener un directorio como texto y convertirlo a tipo Path para unir directorios, buscar archivos, etc.
        '''
        self.base_dir = Path(base_dir)
        # Definir la ruta completa para leer cada archivo
        self.file_mx = self.base_dir.joinpath(file_mx)
        self.file_cl = self.base_dir.joinpath(file_cl)
        # Verificar que existe el archivo en el directorio
        for file_path in [self.file_mx, self.file_cl]:
            if not file_path.is_file():
                file_name = ''.join(file_path.split('/')[-1])
                print(f'Debería haber un archivo llamado: {file_name} en:\n{self.base_dir}\n\nAgrega este archivo e intenta de nuevo!\n')


    def cool_print(self, text: str, sleep_time: float=0.01) -> None: 
        '''
        Imprimir como si se fuera escribiendo
        '''
        acum = ''
        for x in text: 
            # Acumular texto
            acum += x
            # Limpiar pantalla
            clear_output(wait=True)
            # Esperar un poco para emular efecto de escritura
            sleep(sleep_time)
            # Imprimir texto acumulado
            print(acum)
        # Mantener el texto en pantalla
        sleep(1)


    def get_csv(self, file_path, **kwargs) -> DataFrame: 
        '''
        Obtener tabla a partir de un archivo .csv
        '''
        file_name = ''.join(str(file_path).split('/')[-1])
        try: 
            df = read_csv(file_path, low_memory=False, **kwargs)
            # Obtener el número de renglones y columnas para informar al usuario
            df_shape = df.shape
            self.cool_print(f'Archivo con nombre {file_name} fue encontrado en:\n{self.base_dir}\nCon {df_shape[0]} renglones y {df_shape[-1]} columnas')
            df.columns = map(lambda x: str(x).strip().replace(' ','_').lower(), df.columns)
            return df
        # Informar que hubo error al intentar importar el csv
        except: self.cool_print(f'No se encontró el archivo con nombre {file_name} en:\n{self.base_dir}\nSi el archivo csv existe, seguramente tiene un encoding y/o separador diferente a "utf-8" y "," respectivamente\nIntenta de nuevo!')
    

    def export_csv(self, df: DataFrame, file_name: str, name_suffix=None, **kwargs) -> None: 
        '''
        Exportar un archivo en formato csv
        '''
        export_name = f'{file_name}.csv' if name_suffix==None else f'{file_name}_{name_suffix}.csv'
        df.to_csv(self.base_dir.joinpath(export_name), **kwargs)
        self.cool_print(f'Archivo: {export_name} fue exportado exitosamente en:\n{self.base_dir}')


    def clean_text(self, text: str, pattern: str="[^a-zA-Z0-9\s]", lower: bool=False) -> str: 
        '''
        Limpieza de texto
        '''
        # Reemplazar acentos: áàäâã --> a
        clean = normalize('NFD', str(text).replace('\n', ' \n ')).encode('ascii', 'ignore')
        # Omitir caracteres especiales !"#$%&/()=...
        clean = sub(pattern, ' ', clean.decode('utf-8'), flags=UNICODE)
        # Mantener sólo un espacio
        clean = sub(r'\s{2,}', ' ', clean.strip())
        # Minúsculas si el parámetro lo indica
        if lower: clean = clean.lower()
        # Si el registro estaba vacío, indicar nulo
        if clean in ('','nan'): clean = nan
        return clean


    def choose_correct(self, df: DataFrame, col: str, correct_list: list, fill_value: str='DESCONOCIDO', **kwargs) -> DataFrame:
        '''
        Recibe un DataFrame y una lista de posibilidades, especificando la columna a revisar
        elige la opción que más se parezca a alguna de las posibilidades
        '''
        correct_list = list(set(correct_list))
        # Aplicar limpieza de texto a la lista de posibilidades
        correct_clean = list(map(lambda x: self.clean_text(x, lower=True), correct_list))
        # Hacer un diccionario de posibilidades limpias y las originales recibidas
        correct_dict = dict(zip(correct_clean, correct_list))

        # Aplicar la limpieza a la columna especificada
        df[f'{col}_correct'] = df[col].map(lambda x: self.clean_text(x,lower=True))
        # Encontrar las posibilidades más parecidas
        df[f'{col}_correct'] = df[f'{col}_correct'].map(lambda x: get_close_matches(x, correct_clean, **kwargs))
        # Si existen parecidas, traer la primera opción que es la más parecida
        df[f'{col}_correct'] = df[f'{col}_correct'].map(lambda x: x[0] if isinstance(x,list) and len(x)>0 else nan)
        # Regresar del texto limpio a la posibilidad original, lo no encontrado se llena con "fill_value"
        df[f'{col}_correct'] = df[f'{col}_correct'].map(correct_dict).fillna(fill_value)
        return df


    def compare_catalog(self, id_cols: list=['sku','barcodes'], name_cols: list=['name'], export_result: bool=True, n:int=100) -> DataFrame:
        df_mx = self.get_csv(self.file_mx)[id_cols+name_cols].sample(n)
        df_cl = self.get_csv(self.file_cl)[id_cols+name_cols]
        
        df = DataFrame()
        to_omit = [-1]
        for id_col in id_cols:
            aux = df_cl.rename({id_col:f'{id_col}_found'}, axis=1)
            aux['is_found'] = id_col
            without_omit = df_mx.loc[df_mx.index.isin(to_omit)].copy()
            to_append = without_omit.merge(aux, left_on=id_col, right_on=f'{id_col}_found', suffixes=('','_found'))
            df = df.append(to_append, ignore_index=False)
            to_omit.append(to_append.index)

        for name_col in name_cols:
            aux = df_cl.rename({name_col:f'{name_col}_found'}, axis=1)
            aux['is_found'] = name_col
            without_omit = df_mx.loc[df_mx.index.isin(to_omit)].copy()
            to_find = self.choose_correct(without_omit, name_col, df_cl[name_col], n=1, cutoff=0.85)
            to_append = without_omit.merge(to_find, left_on=name_col, right_on=f'{name_col}_correct', how='left', suffixes=('', f'_search_by_{name_col}'))
            to_append.drop(f'{name_col}_correct', axis=1, inplace=True)
            df = df.append(to_append, ignore_index=False)
            to_omit.append(to_append.index)

        return df
        # if export_result: self.export_csv(acum, file_name='Decathlon', index=False, sep='\t', encoding='utf-16')
        # return acum

## Transformar

In [36]:
df = CompareCatalog(BASE_DIR, FILE_MX, FILE_CL).compare_catalog()
df.sample(7)

Archivo con nombre Decathlon_CL.csv fue encontrado en:
/Users/efraflores/Desktop/EF/Corner/Catalog/Decathlon
Con 17552 renglones y 12 columnas


ValueError: a must be greater than 0 unless no samples are taken

In [21]:
df['is_found'].value_counts()

sku    86
Name: is_found, dtype: int64